In [1]:
from load_dataset import load_datasets
import os
import torch
import torch.nn as nn
from utils.one_trial import LitModel
import torch.nn.functional as F
import pandas as pd
from utils.metrics import calc_metric


import pandas as pd


def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha=[0.42, 0.01, 0.76, 0.64, 0.63, 0.66],
    omega=[1.3, 1.0, 0.9, 1.4, 1.2, 1.4],
    gamma: float = 0,
) -> torch.Tensor:
    targets = targets.view_as(inputs).type_as(inputs)
    p = torch.sigmoid(inputs).detach()
    p_t = p * targets + (1 - p) * (1 - targets)
    weights = (1 - p_t) ** gamma
    # target (B, C), alpha (1, C)
    alpha = torch.tensor(alpha, device=inputs.device).type_as(inputs).reshape(1, inputs.shape[1])
    omega = torch.tensor(omega, device=inputs.device).type_as(inputs).reshape(1, inputs.shape[1])
    weights *= alpha * targets + (1 - alpha) * (1 - targets)
    weights *= omega
    loss = F.binary_cross_entropy_with_logits(inputs, targets, weights, reduction="none")
    # loss B x C
    loss = loss.sum() / weights.sum()
    return loss


from utils.one_trial import LitModel
from models.EDCLoc import classifier
import os


def load_model():
    model_params = classifier.get_model_params()
    hparams = classifier.get_hparams()
    hparams["loss_func"] = sigmoid_focal_loss
    hparams["loss_func_hparams"] = dict(
        alpha=[0.42, 0.01, 0.76, 0.64, 0.63, 0.66], omega=[1.3, 1.0, 0.9, 1.4, 1.2, 1.4]
    )
    return LitModel(classifier, hparams, model_params)


def model_predict():
    model = load_model()
    all_test_df = []
    for i in range(5):
        _, _, testset = load_datasets(i)
        all_testres = []
        for j in range(6):
            ckpt_path = os.path.join("save_ckpts", f"fold{i}", f"S{j}.ckpt")
            y_score_test = model.predict_proba(testset, ckpt_path=ckpt_path)
            testres = calc_metric(testset.y[:, j], y_score_test[:, j])
            all_testres.append(testres)
        test_df = pd.DataFrame(all_testres).reset_index(drop=False)
        test_df["fold"] = i
        all_test_df.append(test_df)
    return pd.concat(all_test_df, ignore_index=True)


In [ ]:
test_res = model_predict()

In [3]:
test_res.groupby("index").mean()

,Sn,Sp,Acc,Recall,MCC,Precision,F1,AP,AUC,fold
index,,,,,,,,,,
0,0.783712,0.634509,0.737346,0.783712,0.408041,0.826866,0.804107,0.879968,0.781230,2.0
1,0.797991,0.675868,0.797027,0.797991,0.104457,0.996848,0.885272,0.997508,0.783222,2.0
2,0.414296,0.894139,0.829440,0.414296,0.299527,0.384904,0.395803,0.340872,0.739604,2.0
3,0.677424,0.734189,0.717102,0.677424,0.388290,0.524635,0.589717,0.570807,0.770798,2.0
4,0.445478,0.882776,0.801279,0.445478,0.334191,0.466299,0.454845,0.438645,0.757968,2.0
5,0.256663,0.936764,0.859358,0.256663,0.221866,0.349154,0.292702,0.254509,0.698036,2.0
